In [6]:
!pip install finquant

In [7]:
import investpy as ip
from datetime import datetime
import numpy as np
from tqdm import tqdm
import pandas as pd
from finquant.portfolio import build_portfolio
import quantstats as qs
import warnings
warnings.filterwarnings('ignore')

In [8]:
today = datetime.today().date().strftime('%d/%m/%Y')
start_of_year = f'1/1/{datetime.today().date().year}'

In [ ]:
def analyse_portfolio(assets,
                      amounts,
                      types = None,
                      rf=0.02,
                      num_sim = 5000,
                      from_date = start_of_year,
                      to_date = today,
                      last_month=False, #overrides from_date and to_date
                      show_current_performance=True, 
                      plot_frontier=True,
                      plot_simulations=True,
                      plot_stocks=True):
    
    if types == None:
        types = ['stock' for i in range(len(assets))]
    
    assert len(assets) == len(amounts) == len(types)
    
    #Create allocation dataframe
    alloc = pd.DataFrame({'Name': assets, 'Allocation': amounts})
    alloc['Allocation'] = alloc.Allocation / sum(alloc.Allocation)
    assert sum(alloc.Allocation)
    
    #Get prices
    prices = pd.DataFrame([], columns = assets)
    for i, s in tqdm(enumerate(assets)):
        if last_month:
            if types[i] == 'stock':
                prices[s] = ip.get_stock_recent_data(s, 'brazil').Close
            elif types[i] == 'etf':
                prices[s] = ip.get_etf_recent_data(s, 'brazil').Close
            else:
                pass
        else:
            if types[i] == 'stock':
                prices[s] = ip.get_stock_historical_data(s, 'brazil', from_date=from_date, to_date=to_date).Close
            elif types[i] == 'etf':
                prices[s] = ip.get_etf_historical_data(s, 'brazil', from_date=from_date, to_date=to_date).Close
            else:
                pass
    
    #Build portfolio
    pf = build_portfolio(data=prices, pf_allocation = alloc)
    pf.risk_free_rate = rf
    
    if show_current_performance:
        pf.properties()
    
    print('\n==== Optimal Portfolio ====\n')
    w, res = pf.mc_optimisation(num_trials=num_sim)
    
    print('-- weights --')    
    df1 = w.transpose().reset_index().rename(columns={'index':'Name'})
    df2 = alloc
    df2.columns = ['Name','Current Allocations']
    df3 = pd.merge(df1, df2)
    df3 = df3.set_index('Name')
    print(df3)
    
    print('\n-- performance --')
    print(res)
    
    if plot_frontier:
        
        if plot_simulations:
            pf.mc_plot_results()
        
        pf.ef_plot_efrontier()
        pf.ef.plot_optimal_portfolios()
        pf.plot_stocks()
    
    if plot_stocks:
        pf.comp_cumulative_returns().plot().axhline(y = 0, color = "black", lw = 3);


In [ ]:
#== Felipe ==#
stocks = ['PETR4', 'MGLU3', 'BBAS3', 'CYRE3', 'Fundo de Invest Ishares SP 500']
amounts = [600, 400, 300, 200, 190]
types = ['stock','stock','stock','stock','etf']
analyse_portfolio(stocks, amounts, types, from_date='15/03/2020')

In [2]:
#== Carol ==#
stocks = ['PETR4', 'VVAR3']
amounts = [1000, 1000]
analyse_portfolio(stocks, amounts, plot_simulations=True, from_date='1/10/2020')

NameError: name 'analyse_portfolio' is not defined

In [3]:
#== Felipe com acao de maior impacto no indice de Sortino ==#
stocks = ['PETR4', 'MGLU3', 'BBAS3', 'CYRE3', 'Fundo de Invest Ishares SP 500', 'BIDI11']
amounts = [600, 400, 300, 200, 120, 100]
types = ['stock','stock','stock','stock','etf', 'stock']
analyse_portfolio(stocks, amounts, types, from_date='15/03/2020')

NameError: name 'analyse_portfolio' is not defined